# KML Track Visualization

This notebook processes and visualizes KML files from my South America Expedition.

In [ ]:
# Import libraries
import os
import xml.etree.ElementTree as ET
from datetime import datetime
import re
import folium
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np

## Process KML Files

Here we'll process KML files by adding datetime information to folder names.

In [ ]:
def add_datetime_to_folder_names(input_kml_path, output_kml_path=None):
    """
    Process a KML file to add datetime information to folder names.
    """
    # Parse the KML file
    tree = ET.parse(input_kml_path)
    root = tree.getroot()
    
    # Define the namespace
    ns = {'kml': 'http://www.opengis.net/kml/2.2'}
    
    # Find all Folder elements
    for folder in root.findall('.//kml:Folder', ns):
        # Find the first Placemark in this folder
        first_placemark = folder.find('.//kml:Placemark', ns)
        if first_placemark is not None:
            # Find the TimeStamp/when element
            timestamp_elem = first_placemark.find('.//kml:TimeStamp/kml:when', ns)
            if timestamp_elem is not None and timestamp_elem.text:
                # Extract the date and time
                datetime_str = timestamp_elem.text
                try:
                    # Parse the datetime
                    dt = datetime.strptime(datetime_str, '%Y-%m-%dT%H:%M:%SZ')
                    
                    # Format as YYYY-MM-DDThh
                    folder_date_str = dt.strftime('%Y-%m-%dT%H')
                    
                    # Get the current folder name
                    name_elem = folder.find('./kml:name', ns)
                    if name_elem is not None:
                        current_name = name_elem.text
                        
                        # Check if the name already has a date format at the beginning
                        if not re.match(r'^\d{4}-\d{2}-\d{2}T\d{2}', current_name):
                            # Add the name attribute if it doesn't exist
                            if 'name' not in folder.attrib:
                                folder.attrib['name'] = folder_date_str
                            
                            # Set the folder name element text to include the date
                            name_elem.text = f"{folder_date_str} {current_name}"
                except ValueError:
                    print(f"Could not parse datetime from: {datetime_str}")
    
    # Write the modified KML to the output file if specified
    if output_kml_path:
        tree.write(output_kml_path, encoding='utf-8', xml_declaration=True)
        print(f"Modified KML saved to: {output_kml_path}")
    
    return tree

# Example usage (uncomment when ready)
# kml_file = "your-track.kml"
# output_file = "processed-track.kml"
# tree = add_datetime_to_folder_names(kml_file, output_file)

## Visualize KML Data

Now we'll create interactive maps of the track data.

In [ ]:
def visualize_with_folium(kml_file_path):
    """
    Visualize KML file with Folium - creates an interactive map.
    """
    # Load KML file as a GeoDataFrame
    gdf = gpd.read_file(kml_file_path, driver='KML')
    
    # Extract all coordinates to determine the center of the map
    all_lats = []
    all_lons = []
    
    for geom in gdf.geometry:
        if hasattr(geom, 'xy'):
            lons, lats = geom.xy
            all_lons.extend(lons)
            all_lats.extend(lats)
        elif hasattr(geom, 'geoms'):
            for g in geom.geoms:
                if hasattr(g, 'xy'):
                    lons, lats = g.xy
                    all_lons.extend(lons)
                    all_lats.extend(lats)
    
    if all_lats and all_lons:
        center_lat = sum(all_lats) / len(all_lats)
        center_lon = sum(all_lons) / len(all_lons)
    else:
        # Default to a central location in Colombia (based on your journey order)
        center_lat = 5.5
        center_lon = -75.5
    
    # Create a map centered at the mean point
    m = folium.Map(location=[center_lat, center_lon], zoom_start=9)
    
    # Add each feature to the map
    for idx, row in gdf.iterrows():
        name = row.get('Name', f'Feature {idx}')
        
        if row.geometry.geom_type == 'Point':
            folium.Marker(
                location=[row.geometry.y, row.geometry.x],
                popup=name
            ).add_to(m)
        elif row.geometry.geom_type in ['LineString', 'MultiLineString']:
            # For LineString (track logs)
            folium.GeoJson(
                row.geometry,
                name=name,
                style_function=lambda x: {'color': 'blue', 'weight': 3}
            ).add_to(m)
    
    # Add a layer control
    folium.LayerControl().add_to(m)
    
    return m

# Example usage (uncomment when ready)
# map = visualize_with_folium("processed-track.kml")
# map

## Create Elevation Profile

Extract elevation data from the KML file and create a profile visualization.

In [ ]:
def create_elevation_profile(kml_file_path):
    """
    Extract elevation data from KML and create a profile chart.
    """
    # Load KML file
    gdf = gpd.read_file(kml_file_path, driver='KML')
    
    # Extract LineString geometries (tracks)
    lines = [row.geometry for idx, row in gdf.iterrows() 
             if row.geometry.geom_type in ['LineString', 'MultiLineString']]
    
    # Create a figure
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # Process each track
    for i, line in enumerate(lines):
        if hasattr(line, 'coords'):
            # For LineString
            coords = np.array(list(line.coords))
            if coords.shape[1] >= 3:  # Check if we have elevation data
                # Calculate distance along track
                distances = [0]
                for j in range(1, len(coords)):
                    # Simple Euclidean distance (not geodesic)
                    dist = np.sqrt(np.sum((coords[j, :2] - coords[j-1, :2])**2))
                    distances.append(distances[-1] + dist)
                
                # Plot elevation vs. distance
                ax.plot(distances, coords[:, 2], label=f'Track {i+1}')
        
        elif hasattr(line, 'geoms'):
            # For MultiLineString
            for j, segment in enumerate(line.geoms):
                coords = np.array(list(segment.coords))
                if coords.shape[1] >= 3:  # Check if we have elevation data
                    # Calculate distance along track segment
                    distances = [0]
                    for k in range(1, len(coords)):
                        dist = np.sqrt(np.sum((coords[k, :2] - coords[k-1, :2])**2))
                        distances.append(distances[-1] + dist)
                    
                    # Plot elevation vs. distance
                    ax.plot(distances, coords[:, 2], label=f'Track {i+1}, Segment {j+1}')
    
    # Add labels and title
    ax.set_xlabel('Distance (arbitrary units)')
    ax.set_ylabel('Elevation (m)')
    ax.set_title('Elevation Profile')
    ax.grid(True)
    
    # Add legend if there are multiple tracks
    if len(lines) > 1:
        ax.legend()
    
    return fig

# Example usage (uncomment when ready)
# elevation_fig = create_elevation_profile("processed-track.kml")
# elevation_fig